In [87]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import pickle
import json
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("synthetic")

LangSmith 추적을 시작합니다.
[프로젝트명]
synthetic


In [26]:
with open("outputs/split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [28]:
len(split_documents)

109

In [79]:
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0)

In [ ]:
class SyntheticData(BaseModel):
    """Synthetic data with details."""
    query: str = Field(..., description="The query of the data")
    answers: str = Field(..., description="The answers of the data")
    page_number: str = Field(..., description="The page number of the evidence")
    evidence: str = Field(..., description="The evidence of the data")
 
 

In [43]:
trimmed = split_documents[2:]

In [82]:
len(trimmed), len(split_documents)

(107, 109)

In [83]:
system_prompt = "You are a careful dataset generator for RAG. Only answer from the provided passage."
    
user_prompt = f"""
            Task: Extract 3 atomic facts and write 1 QA whose answer from following document 
            Response: query : question, answers : answer, evidence : page content for the answer
        """

queries = []
for doc in trimmed:
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "user", "content": "page number: " + str(doc.metadata["page"])},
        {"role": "user", "content": "content : " + doc.page_content},
    ]
    queries.append(prompt)



In [84]:
model_with_structure = llm.with_structured_output(SyntheticData)
responses = model_with_structure.batch(queries)

In [85]:
len(responses)

107

In [86]:
arr = []
for r in responses:
    obj = {
        "query": r.query,
        "answers": r.answers,
        "evidence": r.evidence,
        "page_number": r.page_number,
    }
    arr.append(obj)



In [88]:
with open("outputs/synthetic_out.jsonl", "w", encoding="utf-8") as f:
    for r in arr:
        f.write(json.dumps(r, ensure_ascii=False, indent=2) + "\n")

In [ ]:
import pandas as pd
df = pd.DataFrame(arr)
df.to_csv("outputs/synthetic_out.csv", index=True)

In [94]:
# To get 'query' and 'answers' in a specific row (e.g., row number i), use:
i = 15  # Change this to your desired row index
query = df.loc[i, "query"]
answers = df.loc[i, "answers"]
print("Query:", query)
print("Answers:", answers)


Query: What should central banks do to manage price stability according to the document?
Answers: 1) Central banks should remain vigilant and react promptly to shifts in the balance of risks to price stability. 2) Provided inflation expectations remain well anchored, policy interest rate reductions should continue in economies in which underlying inflation is projected to moderate towards target. 3) Maintaining central bank independence will preserve policy credibility and reduce the volatility and persistence of inflation.
